In [460]:
import pandas as pd
import networkx as nx
from collections import Counter
import ast
import re
import math
import numpy as np
from os import listdir

In [461]:
#df = pd.read_json('C:/Users/lamersws1/Documents/Data/firstdata.json')

#df = pd.read_csv('C:/Users/lamersws1/Documents/GitHub/data_opensource.csv')
df['user'] = df['user'].apply(lambda x: ast.literal_eval(x))
df['entities'] = df['entities'].apply(lambda x: ast.literal_eval(x))
datadir = 'C:/Users/lamersws1/Documents/GitHub'
df = pd.concat([pd.read_json(datadir+'/'+file) for file in [x for x in listdir(datadir) if x[-5:] == ".json"]], axis=0, sort=False)

In [509]:
df_100 = df.head(100)
df_100.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,limit,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries,open source,#opensource
5,NaN,None,2018-05-25 15:03:59,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",None,None,0.0,0.0,low,None,1.000030e+18,1.000030e+18,None,NaN,NaN,NaN,NaN,0.0,en,None,None,NaN,0.0,None,NaN,NaN,None,0.0,0.0,0.0,{'created_at': 'Fri May 25 13:17:49 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @SpicyFiles: Whoopsie Daisy here’s your ope...,2018-05-25 15:03:59.578,0.0,"{'id': 3166264359, 'id_str': '3166264359', 'na...",None,1,0
10,NaN,None,2018-05-25 15:04:04,"[25, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,{'full_text': '@AaronHawryluk @elonmusk You di...,0.0,0.0,low,None,1.000030e+18,1.000030e+18,AaronHawryluk,9.997682e+17,9.997682e+17,133435056.0,133435056.0,0.0,en,None,None,NaN,0.0,None,NaN,NaN,None,0.0,0.0,0.0,None,"<a href=""http://twitter.com/download/android"" ...",@AaronHawryluk @elonmusk You didn't even let h...,2018-05-25 15:04:04.847,1.0,"{'id': 734197130, 'id_str': '734197130', 'name...",None,1,0
11,NaN,None,2018-05-25 15:04:04,"[0, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,{'full_text': 'Congrats to all three! And you ...,0.0,0.0,low,None,1.000030e+18,1.000030e+18,None,NaN,NaN,NaN,NaN,1.0,en,None,None,0.0,0.0,{'created_at': 'Fri May 25 14:57:34 +0000 2018...,1.000028e+18,1.000028e+18,"{'url': 'https://t.co/POciowUQh5', 'expanded':...",0.0,0.0,0.0,None,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Congrats to all three! And you can follow @far...,2018-05-25 15:04:04.924,1.0,"{'id': 14286491, 'id_str': '14286491', 'name':...",None,1,0
14,NaN,None,2018-05-25 15:04:28,None,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",None,{'full_text': 'Jogue Super Mario Bros no Ubunt...,0.0,0.0,low,None,1.000030e+18,1.000030e+18,None,NaN,NaN,NaN,NaN,0.0,pt,None,None,0.0,0.0,None,NaN,NaN,None,0.0,0.0,0.0,None,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Jogue Super Mario Bros no Ubuntu com o SuperTu...,2018-05-25 15:04:28.312,1.0,"{'id': 16219932, 'id_str': '16219932', 'name':...",None,1,0
15,NaN,None,2018-05-25 15:04:29,None,"{'hashtags': [{'text': 'ODSC', 'indices': [121...",None,None,0.0,0.0,low,None,1.000030e+18,1.000030e+18,None,NaN,NaN,NaN,NaN,0.0,en,None,None,0.0,0.0,None,NaN,NaN,None,0.0,0.0,0.0,{'created_at': 'Fri May 25 14:01:19 +0000 2018...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @odsc: ODSC East 2018 – Open Source Data Sc...,2018-05-25 15:04:29.255,0.0,"{'id': 2397407502, 'id_str': '2397407502', 'na...",None,1,0


In [463]:
df.shape

(100000, 38)

In [464]:
df['entities'].isnull().sum()

1

In [465]:
df = df[df['entities'].isnull() == False]

In [466]:
sum(df['text'].str.contains('open source'))

7580

In [467]:
hashtags = u"open source, #opensource"
hashtags_list = hashtags.split(", ")
hashtags_list

['open source', '#opensource']

In [468]:
for word in hashtags_list:
    print(word)
    df[word] = df['text'].str.lower().str.contains(word).astype(int)
test = df.iloc[:1000]

open source
#opensource


In [469]:
df = df[(df['open source']>=1)|(df['#opensource']>=1)]

In [470]:
df.to_json('open_source_data.json')

ValueError: DataFrame index must be unique for orient='columns'.

Explore available collumns

In [325]:
list(df)

['contributors',
 'coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'limit',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'timestamp_ms',
 'truncated',
 'user',
 'withheld_in_countries',
 'open source',
 '#opensource']

In [326]:
df.describe()

,contributors,favorite_count,favorited,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,possibly_sensitive,quote_count,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,truncated,open source,#opensource
count,0.0,31835.0,31835.0,3.183500e+04,3.183500e+04,8.830000e+02,8.830000e+02,9.950000e+02,9.950000e+02,31835.000000,7491.000000,31835.0,9.010000e+02,9.010000e+02,31835.0,31835.0,31835.0,31835.000000,31835.000000,31835.000000
mean,NaN,0.0,0.0,1.000938e+18,1.000938e+18,1.000165e+18,1.000165e+18,1.715042e+17,1.715042e+17,0.028302,0.010012,0.0,9.985456e+17,9.985456e+17,0.0,0.0,0.0,0.082645,0.985833,0.043788
std,NaN,0.0,0.0,3.255410e+14,3.255410e+14,6.764598e+15,6.764598e+15,3.473333e+17,3.473333e+17,0.165837,0.099565,0.0,8.177063e+15,8.177063e+15,0.0,0.0,0.0,0.275349,0.118180,0.204627
min,NaN,0.0,0.0,1.000030e+18,1.000030e+18,8.702325e+17,8.702325e+17,2.460000e+02,2.460000e+02,0.000000,0.000000,0.0,9.052803e+17,9.052803e+17,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,NaN,0.0,0.0,1.000930e+18,1.000930e+18,1.000274e+18,1.000274e+18,3.668982e+07,3.668982e+07,0.000000,0.000000,0.0,9.999411e+17,9.999411e+17,0.0,0.0,0.0,0.000000,1.000000,0.000000
50%,NaN,0.0,0.0,1.000993e+18,1.000993e+18,1.000773e+18,1.000773e+18,3.444253e+08,3.444253e+08,0.000000,0.000000,0.0,1.000055e+18,1.000055e+18,0.0,0.0,0.0,0.000000,1.000000,0.000000
75%,NaN,0.0,0.0,1.001097e+18,1.001097e+18,1.001181e+18,1.001181e+18,3.100563e+09,3.100563e+09,0.000000,0.000000,0.0,1.000525e+18,1.000525e+18,0.0,0.0,0.0,0.000000,1.000000,0.000000
max,NaN,0.0,0.0,1.001553e+18,1.001553e+18,1.001551e+18,1.001551e+18,1.001153e+18,1.001153e+18,1.000000,1.000000,0.0,1.001530e+18,1.001530e+18,0.0,0.0,0.0,1.000000,1.000000,1.000000


Let's look at hashtags

In [382]:
test = df[[x['id']==2548144838 for x in df['user']]]
print(len(test))
test = df[[x==2548144838 for x in df['in_reply_to_screen_name']]]
print(len(test))
test = df[[x['user']['id']==2548144838 if isinstance(x, dict) else False for x in df['quoted_status']]]
print(len(test))


0
0
0


Remove this useless record from the data

In [425]:
def extract_anomaly(df):

    out = []

    for index, row in df.iterrows():

        if isinstance(row.user, dict):
        
            usermentions = row.entities['user_mentions']
            
            cnt = 0
            
            for u in usermentions:
                if u['id'] == 2548144838:
                    cnt += 1
            out.append(cnt)
    

    return out

test = df.copy()
test['this']=extract_anomaly(df)
test = test[test['this'] > 0]

Let's look at the mentions network

In [537]:
def extract_edges(df, what = ['mentions']):

    out = []

    for index, row in df.iterrows():

        if isinstance(row.user, dict):
        
            assert isinstance(row.user, dict), "user is not dict in {}".format(index)
            assert isinstance(row.entities, dict), "entities is not dict in {}".format(index)
            
            if 'mentions' in what:
                usermentions = row.entities['user_mentions']

                for u in usermentions:
                    if u['id'] != 2548144838:
                        out.append((row.user['id'], u['id']))
            
            if 'replies' in what:
                u = row.in_reply_to_user_id

                if np.isnan(u) == False:
                    out.append((row.user['id'], int(u)))
                    
            if 'quotes' in what:
                u = row.quoted_status
                
                if isinstance(u, dict):
                    out.append((row.user['id'], u['user']['id']))
                    
            if 'retweets' in what:
                u = row.retweeted_status
                
                if isinstance(u, dict):
                    if u['user']['id'] != 2548144838:
                        out.append((row.user['id'], u['user']['id']))
                
    
    out = [o for o in out if o[0]!=o[1]]
    
    return out

In [538]:
edges = extract_edges(df, ['mentions', 'replies', 'quotes', 'retweets'])
len(edges)

19719

In [539]:
edges

[(3166264359, 975548535067348992),
 (3166264359, 975548535067348992),
 (734197130, 133435056),
 (734197130, 44196397),
 (734197130, 133435056),
 (14286491, 1138409204),
 (14286491, 774982216870981632),
 (14286491, 20624794),
 (2397407502, 3018841323),
 (2397407502, 3018841323),
 (946788839204339712, 2892556445),
 (946788839204339712, 2892556445),
 (37961608, 821794726282203138),
 (37961608, 821794726282203138),
 (128722302, 112625891),
 (128722302, 112625891),
 (946788839204339712, 2892556445),
 (946788839204339712, 2892556445),
 (980052164893650944, 835484220671815681),
 (980052164893650944, 835484220700000128),
 (873550863121371138, 695685504113766400),
 (873550863121371138, 695685504113766400),
 (27836117, 14169633),
 (27836117, 14169633),
 (937754245952163840, 495063100),
 (937754245952163840, 840809649901264896),
 (937754245952163840, 495063100),
 (2824958945, 117142912),
 (2824958945, 117142912),
 (88450167, 930021796253978624),
 (735555667965136897, 88450167),
 (7355556679651368

In [540]:
temp = df.head(2)

In [541]:
len(list(set([x[0] for x in edges] + [x[1] for x in edges])))

9557

In [542]:
from itertools import chain

In [543]:
Counter(chain(*edges))[2548144838]

0

In [544]:
users[2548144838]

{'screen_name': 'likalo_llc'}

In [545]:
def extract_users(df):
    
    users = [(u['id'], u['screen_name'], u['name']) for u in df['user'] if isinstance(u, dict)]
    users = list(set(users))
    
    user_mentions = [u['user_mentions'] for u in df['entities'] if isinstance(u, dict)]
    for l in user_mentions:
        for u in l:
            users.append((u['id'], u['screen_name']))
    
    user_quotes = [u for u in df['quoted_status'] if isinstance(u, dict)]
    for u['user'] in user_quotes:
        users.append((u['id'], u['screen_name']))
    
    user_replies = [(int(x[1]), x[0]) for x in zip(df['in_reply_to_screen_name'], df['in_reply_to_user_id']) if isinstance(x[0], str)]
    
    users = users + user_replies
    
    users = list(set(users))
    
    users = {u[0]: {'screen_name': u[1]} for u in users}
    
    return users


In [546]:
users = extract_users(df)
users[2548144838]

{'screen_name': 'likalo_llc'}

In [547]:
def user_count_tweets(df):
    users = [u['id'] for u in df['user'] if isinstance(u, dict)]
    
    users = Counter(users)
    
    return {x[0]: {'n_tweets': x[1]} for x in users.most_common()}

user_tweetcounts = user_count_tweets(df)

In [548]:
len(edges_w)

29955

In [549]:
#add weights to edges
edges_w = Counter(edges)
edges_w = [(e[0], e[1], edges_w[e]) for e in edges_w]

In [550]:
G=nx.Graph()
G.add_edges_from(edges)
nx.set_node_attributes(G, users)
nx.set_node_attributes(G, user_tweetcounts)
nx.set_node_attributes(G, {node: {'degree_original': val} for node, val in list(G.degree)})
len(G)

9557

Get giant component, discard rest

Get only those nodes with degree > 25, then get giant component of those

In [551]:
GC = sorted(nx.connected_components(G), key=len, reverse=True)[0]
SG = G.subgraph(GC)
top_connected_nodes = [x[0] for x in SG.degree() if x[1] >= 1]
SG = SG.subgraph(top_connected_nodes)
SG = SG.subgraph(sorted(nx.connected_components(SG), key=len, reverse=True)[0])

In [552]:
len(SG.nodes)

4260

In [553]:
#nx.draw_networkx(SG)

In [554]:
#SG.nodes

In [555]:
relabeldict = {key: val['screen_name'] for key, val in users.items()}

In [556]:
nx.write_graphml(nx.relabel_nodes(SG, relabeldict),'mentions_network.graphml')

In [458]:
#nx.draw_networkx(nx.relabel_nodes(SG, relabeldict))

In [459]:
#nx.write_graphml(nx.relabel_nodes(G.subgraph(GC), relabeldict),'mentions_network_full.graphml')